## 层与块

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))

X = torch.rand(2,20)
net(X)

tensor([[-0.2018, -0.1770,  0.0759, -0.0113,  0.0695, -0.0691,  0.1798,  0.1663,
          0.1354,  0.2232],
        [-0.1788, -0.2247,  0.2187, -0.0771,  0.0275, -0.0590,  0.2704,  0.1329,
          0.0477,  0.1737]], grad_fn=<AddmmBackward0>)

### 自定义块

In [2]:
class MLP(nn.Module):
    # 用模型参数声明层。这里，我们声明两个全连接的层
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20,256)
        self.out = nn.Linear(256,10)

    def forward(self,X):
        return self.out(F.relu(self.hidden(X)))

In [3]:
net = MLP()
net(X)

tensor([[ 0.2348,  0.1469, -0.2153, -0.1538,  0.3111, -0.0428, -0.0213,  0.1795,
          0.0738, -0.0620],
        [ 0.2202,  0.0319, -0.1629, -0.1038,  0.2977, -0.0259, -0.1014,  0.1379,
          0.0845,  0.0200]], grad_fn=<AddmmBackward0>)

### 顺序块

In [6]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx,module in enumerate(args):
            self._modules[str(idx)] = module

    def forward(self,X):
        for block in self._modules.values():
            X = block(X)
        return X

In [7]:
net = MySequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))
net(X)

tensor([[ 0.1709,  0.0578,  0.0107, -0.2217, -0.0930, -0.1651, -0.1133, -0.1400,
         -0.0635, -0.1261],
        [ 0.1209,  0.0063,  0.0809, -0.1548,  0.0258, -0.0605, -0.1356, -0.0966,
         -0.1201,  0.0238]], grad_fn=<AddmmBackward0>)

In [8]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20,20),requires_grad=False)
        self.linear = nn.Linear(20,20)

    def forward(self,X):
        X = self.linear(X)
        X = F.relu(torch.mm(X,self.rand_weight)+1)
        X = self.linear(X)
        while X.abs().sum()>1:
            X/=2
        return X.sum()

In [9]:
net = FixedHiddenMLP()
net(X)

tensor(0.1275, grad_fn=<SumBackward0>)

In [11]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20,64),nn.ReLU(),
                                 nn.Linear(64,32),nn.ReLU())
        self.linear = nn.Linear(32,16)

    def forward(self,X):
        return self.linear(self.net(X))
    
chimera = nn.Sequential(NestMLP(),nn.Linear(16,20),FixedHiddenMLP())
chimera(X)

tensor(-0.0121, grad_fn=<SumBackward0>)